### Library

In [12]:
import os
from dotenv import load_dotenv

from binance.client import Client

import pandas as pd
import numpy as np
from time import sleep
from datetime import datetime
import ta

from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting.test import SMA

### TA Calculation Functions

#### SMA

In [13]:
def sma(df, window):
    sma = ta.trend.SMAIndicator(pd.Series(df), window=window).sma_indicator()
    return sma

#### RSI

In [14]:
def rsi(df, window=14):
    rsi = ta.momentum.RSIIndicator(pd.Series(df), window=window).rsi()
    return rsi

#### EMA

In [15]:
def ema(df, period=200):
    ema = ta.trend.EMAIndicator(pd.Series(df), window=window).ema_indicator()
    return ema

#### MACD

In [16]:
def macd(df):
    macd = ta.trend.MACD(pd.Series(df)).macd()
    return macd

#### Bollinger Bands

In [17]:
def signal_h(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_hband()
def signal_l(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_lband()

### Strategy Class

#### Modified SMA

In [ ]:
class Modified_SMA(SignalStrategy,
                   TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()

        # Precompute the two moving averages
        sma1 = self.I(sma, self.data.Close, self.n1)
        sma2 = self.I(sma, self.data.Close, self.n2)

        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)

        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95

        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)

        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)
        

#### RSI

In [ ]:
class SMA_RSI(Strategy):
    # Any variables you want:
    n1 = 10
    n2 = 20
    rsi_period = 14
    tp = 0.03
    sl = 0.02
    def init(self):
        # Take close prices as actual price
        price = self.data.Close
        # Declare indicators you will use in the strategy:
        self.rsi = self.I(rsi, self.data.Close, self.rsi_period)
        self.sma1 = self.I(SMA, price, self.n1)
        self.sma2 = self.I(SMA, price, self.n2)

    def next(self):
        price = float(self.data.Close[-1])
        if crossover(self.sma1, self.sma2) and self.rsi[-2] < 30:
            self.buy (tp = (1 + self.tp) * price,
                      sl = (1 - self.sl) * price)
        if crossover(self.sma2, self.sma1) and self.rsi[-2] > 70:
            self.sell(tp = (1 - self.tp) * price,
                      sl = (1 + self.sl) * price)
        

#### Simple Moving Average

In [ ]:
class SMA_RSI_test(Strategy):
    # Any variables you want:
    n1 = 10
    n2 = 15
    rsi_period = 14
    tp = 0.03
    sl = 0.02
    def init(self):
        # Take close prices as actual price
        price = self.data.Close
        # Declare indicators you will use in the strategy:
        self.rsi = self.I(rsi, self.data.Close, self.rsi_period)
        self.sma1 = self.I(SMA, price, self.n1)
        self.sma2 = self.I(SMA, price, self.n2)

    def next(self):
        price = float(self.data.Close[-1])
        if crossover(self.sma1, self.sma2) and self.rsi[-2] < 30:
            self.buy (tp = (1 + self.tp) * price,
                      sl = (1 - self.sl) * price)
        if crossover(self.sma2, self.sma1) and self.rsi[-2] > 70:
            self.sell(tp = (1 - self.tp) * price,
                      sl = (1 + self.sl) * price)
        

### ML Model


## Main

### Binance API

In [6]:
load_dotenv()
API_KEY    = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

client = Client(API_KEY, API_SECRET)

### Parameters

### Fetch Historical Data

In [7]:
symbol = 'BTCUSDT'
interval = '1h'

start_time_train = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_time_train   = int(datetime(2021,12,31,0,0).timestamp() * 1000)
start_time_test  = int(datetime(2022,1,1,0,0).timestamp() * 1000)
end_time_test    = int(datetime(2023,12,31,0,0).timestamp() * 1000)

kline_train = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time_train, end_str=end_time_train)
kline_test  = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time_test , end_str=end_time_test)

columns = ['index','Open', 'High', 'Low', 'Close', 'Volume']

data_train = pd.DataFrame(kline_train)
data_train = data_train.iloc[:, :6]
data_train.columns  = columns
data_train['index'] = pd.to_datetime(data_train['index'], unit='ms')
data_train.set_index('index', inplace=True)
data_train = data_train.astype(float)

data_test = pd.DataFrame(kline_test)
data_test = data_test.iloc[:, :6]
data_test.columns = columns
data_test['index'] = pd.to_datetime(data_test['index'], unit='ms')
data_test.set_index('index', inplace=True)
data_test = data_test.astype(float)

### Backtesting

#### Backtesting

In [ ]:
bt_train = Backtest(data_train, Modified_SMA, cash = 1000000, commission=0.00075)
stats = bt_train.run()
stats

In [ ]:
bt_train = Backtest(data_train, Modified_SMA_test, cash = 1000000, commission=0.00075)
stats = bt_train.optimize(n1=range(5, 30, 5),
                          n2=range(10, 70, 5),
                          maximize='Equity Final [$]',
                          constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats['_strategy']

In [ ]:
bt_test = Backtest(data_test, SMA_RSI_test, cash = 1000000, commission=0.00075)
stats = bt_test.run()
stats

#### Graph

In [ ]:
bt_test.plot()

In [ ]:
stats.tail()

In [ ]:
stats['_equity_curve']

## For Temp Ad Hoc Testing

In [1]:
def get_X(data):
    return data.filter(like='X').values

In [2]:
def get_y(data):
    y = data['Close'].pct_change(48).shift(-48)

In [3]:
def get_clean_Xy(data):
    X = get_X(data)
    y = get_y(data).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [29]:
close = data_train.Close.values
sma10 = sma(data_train.Close, 10).values
sma20 = sma(data_train.Close, 20).values
sma50 = sma(data_train.Close, 50).values
sma100 = sma(data_train.Close, 100).values
upper = signal_h(data_train.Close).values
lower = signal_l(data_train.Close).values

# Design matrix / independent features:

# Price-derived features
data_train['X_SMA10'] = (close - sma10) / close
data_train['X_SMA20'] = (close - sma20) / close
data_train['X_SMA50'] = (close - sma50) / close
data_train['X_SMA100'] = (close - sma100) / close

data_train['X_DELTA_SMA10'] = (sma10 - sma20) / close
data_train['X_DELTA_SMA20'] = (sma20 - sma50) / close
data_train['X_DELTA_SMA50'] = (sma50 - sma100) / close

# Indicator features
data_train['X_MOM'] = data_train.Close.pct_change(periods=2)
data_train['X_BB_upper'] = (upper - close) / close
data_train['X_BB_lower'] = (lower - close) / close
data_train['X_BB_width'] = (upper - lower) / close

# Some datetime features for good measure
data_train['X_day'] = data_train.index.dayofweek
data_train['X_hour'] = data_train.index.hour

data_train = data_train.dropna().astype(float)

In [30]:
data_train

,Open,High,Low,Close,Volume,X_SMA10,X_SMA20,X_SMA50,X_SMA100,X_DELTA_SMA10,X_DELTA_SMA20,X_DELTA_SMA50,X_MOM,X_BB_upper,X_BB_lower,X_BB_width,X_day,X_hour
index,,,,,,,,,,,,,,,,,,
2020-01-04 19:00:00,7306.60,7353.84,7283.01,7334.83,1966.929401,0.001912,0.000839,0.012521,0.016604,-0.001073,0.011682,0.004083,-0.002334,0.004193,-0.005871,0.010065,5.0,19.0
2020-01-04 20:00:00,7334.89,7352.64,7310.67,7341.06,1433.864061,0.002449,0.001423,0.012359,0.017262,-0.001026,0.010936,0.004903,0.004672,0.003371,-0.006216,0.009587,5.0,20.0
2020-01-04 21:00:00,7340.90,7367.31,7330.72,7350.68,869.007514,0.003561,0.002447,0.012652,0.018300,-0.001114,0.010205,0.005648,0.002161,0.002284,-0.007178,0.009462,5.0,21.0
2020-01-04 22:00:00,7350.72,7359.70,7342.67,7350.24,474.926109,0.002993,0.002235,0.011500,0.017994,-0.000758,0.009265,0.006494,0.001251,0.002597,-0.007067,0.009664,5.0,22.0
2020-01-04 23:00:00,7350.05,7363.00,7328.90,7354.11,528.793108,0.002834,0.002617,0.010996,0.018265,-0.000217,0.008379,0.007269,0.000467,0.002359,-0.007593,0.009952,5.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-30 12:00:00,47515.45,47719.37,47400.00,47514.17,2014.693870,0.012095,0.010941,-0.001546,-0.031666,-0.001155,-0.012486,-0.030120,0.011954,0.004207,-0.026088,0.030296,3.0,12.0
2021-12-30 13:00:00,47514.16,47555.55,47300.00,47321.49,1350.130450,0.006345,0.007160,-0.004830,-0.035298,0.000815,-0.011990,-0.030468,-0.004082,0.007420,-0.021740,0.029160,3.0,13.0
2021-12-30 14:00:00,47321.50,47437.72,47166.97,47345.03,1333.652250,0.005524,0.007851,-0.003589,-0.034205,0.002327,-0.011439,-0.030617,-0.003560,0.006187,-0.021888,0.028075,3.0,14.0


## Reference

Backtesting User Manual

https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html